## Trading Algorithm Notes

### Pairs Trading
* If two securities are correlated, when they breakout from each other -> short the high one, buy the low one
* When prices move back together, close position
* Determine correlation using percent change and r-value

### Momentum Strategies
    * It is assumed price move will hold momentum for certain period or time
    * High volatility == Vix > 25
    * Use stop loss that follows the price upward and resets each time the stock price hits a new high
    * Additional functionality:
        1) Protect against downside moves with options trading
            * Buy put options on long positions (only buy 1) after running certain amount higher than previous high
            * Only buy when volatility is high
#### Gap and Go
    * Looks for overnight gappers and bets on continuation of that move
    * Open Short-term long trade and use stop loss
#### Earnings/News
    * Look at the most well known/liquid stocks that had earnings in the past few days
    * Focus on stocks that have an especially negative reaction from the market (-2%)
    * Open long position and use stop loss
#### Breakout Strategy
    * Look at past highs and generate buy signals as soon as price manages to break out from its previous high
    * Use dynamic lookback ranges based on volatility
        * When volatility is high, look further back
        * When low, look less
    * Open short-term long position and use stop loss

### Moving Avg Strategys
#### Look for rollovers
    * If faster MA crosses slower MA in upward motion -> buy
    * If faster MA crosses slower MA in downward motion -> sell
        * Consider adding a momentum threshold so that it doesnt trade too often
        * Momentum is identified by increasing distance between MA's
    * Where does 8-bar MA come in?
    * Decreases that trigger bearish moving average rollovers in multiple time frames offer short sale opportunity

#### Look for sideways markets - causes SMA's to become less accurate
    * Steady avg's
    * Frequent crossovers
    * High volatility, higher trading range    

#### Taking profits
    * Short sale - take profits when price lifts above 5-bar MA
    * Long sale - take profits when price dips below 5-bar MA
    
#### Exponential MA
    * multiplier = 2 ÷ (number of observations + 1)
    * EMA = Closing price x multiplier + EMA (previous day) x (1-multiplier)
    
#### MACD
* MACD=12-Period EMA − 26-Period EMA
* A 9-period EMA of the MACD called the "signal line," is then plotted on top of the MACD line as a trade indicator

## Creating Trading Robot

In [1]:
from config import Alpaca_key,Alpaca_secret_key,Alpaca_endpoint
import alpaca_trade_api as tradeapi
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

# API Auth
api = tradeapi.REST(Alpaca_key,Alpaca_secret_key,Alpaca_endpoint)

### Universe Selection Object

In [2]:
import time

def get_price_dataframe(ticker="SPY",length="5Min",limit=100): # length= 1Min, 5Min, 15Min, or 1D
    """
        Arguements:
            ticker -> Stock ticker symobl (default SPY)
            length -> Amount of time between bars (default 5Min)
            limit -> Number of bars returned (default 100)
            
        Returns:
        Returns a Pandas Dataframe of historical price data for any given ticker
    """
    # Submitting request
    barset=api.get_barset(ticker,length,limit)
    # Creating Dataframe
    data = pd.DataFrame()
    for bar in barset[ticker]:
        close=bar.c
        open=bar.o
        high=bar.h
        low=bar.l
        volume=bar.v
        datetime=bar.t
        if len(data) == 0:
            index = [0]
        else:
            index = [max(data.index)+1]
        new_row = pd.DataFrame({
            "date":datetime,
            "open":open,
            "high":high,
            "low":low,
            "close":close,
            "volume":volume}, index=index)
        data = data.append(new_row)
    return data

def tradeable_technology_stocks():
    """
        Returns:
            List of technology sector ticker symbols that meet filter criteria

        Notes:
            Tickers are gathered from csv files of all Technology stocks that trade on Nasdaq and S&P 500 and meet filter criteria
    """
    # Get Alpaca's stock list
    assets = api.list_assets("active", "us_equity")
    # Filter for TRADEABLE, SHORTABLE on Alapaca's platform
    assets = [asset for asset in assets if asset.tradable == True and asset.shortable == True]
    alpaca_tradable_symbols = [asset.symbol for asset in assets]
    # Filter for US EQUITY, TECHNOLOGY SECTOR, $2-$200 BILLION MARKET CAP, NASDAQ/S&P 500
    SP_tech_stocks = pd.read_csv("S&P_500_tech_assets_barchart_dotcom.csv")
    nasdaq_tech_stocks = pd.read_csv("nasdaq_tech_assets_nasdaq_dotcom.csv")
    combined_tickers = pd.concat([SP_tech_stocks["Symbol"], nasdaq_tech_stocks["Symbol"]], ignore_index=True)
    assets = [ticker for ticker in combined_tickers if ticker in alpaca_tradable_symbols]
    # Returns list of ticker symbols
    return assets

def find_most_liquid_stocks(periods=10, num_returned=10, bar_frequency="1D"):
    """
        Arguements:
            periods -> Number of prior days to use in avg volume calculation
                 -> High Volatility: Look back further, Low Volatility: Look back less
            num_returned -> Number of ticker symbols to return
            bar_frequency -> The amount of time that is included in each period (AKA: bar, row)
            
        Returns:
            List of the most liquid ticker symbols that meet below criteria:
                1) US Equity
                2) In the Technology Sector
                3) Trades on S&P 500 or Nasdaq Exchange
                4) $2-$200 Billion Market Cap
                5) Tradeable and Shortable on Alpaca's trading platform
    """
    # Returns list of technology sector ticker symbols that meet filter criteria
    tradable_stocks = tradeable_technology_stocks()
    
    # Finding the most liquid stocks by $ volume
    stock_volumes = {}
    count = 1
    for ticker in tradable_stocks:
        # Get prior 10 days of data
        data = get_price_dataframe(ticker, bar_frequency, periods)
        ################################# ADD OTHER FILTERS HERE ############################
        # Add avg volume to dictionary
        avg_10_day_volume = data["volume"].mean()
        stock_volumes[ticker] = avg_10_day_volume
        count+=1
        # Alpaca API limit is 200 calls per minute
        if count == 180:
            time.sleep(45)
    # Sort the dictionary, and return the N most liquid stocks 
    return [(ticker) for ticker, volume in sorted(stock_volumes.items(), key=lambda item: item[1])][-num_returned:]

### Alpha Object

In [13]:
import numpy as np

# Identifying 5 bar fractals (fractals remove noise of market swings to better identify key levels)
    # A fractal is a candlestick pattern made by 5 candles. 
    # The third candle has the lowest low price, the previous candles have decreasing lows and the next candles have increasing lows.
    # By this pattern, the low of the third candle is the support level.
    # The same concept applies to resistance levels, where the third candle has the highest high of the five ones.
def isSupport(df,i):
    support = df['low'][i] < df['low'][i-1] and df['low'][i] < df['low'][i+1] and df['low'][i+1] < df['low'][i+2] and df['low'][i-1] < df['low'][i-2]
    return support
def isResistance(df,i):
    resistance = df['high'][i] > df['high'][i-1] and df['high'][i] > df['high'][i+1] and df['high'][i+1] > df['high'][i+2] and df['high'][i-1] > df['high'][i-2]
    return resistance

# Given a price value this function returns False if it is near a previously discovered key level.
# A level is near a previously discovered key level if their distance is less than the average candle size (s) in our chart
def isFarFromLevel(l, levels, s):
    levels = [(index,level) for index,level,category in levels]
    return np.sum([abs(l-x) < s  for x in levels]) == 0

def get_key_levels(tickers_to_trade, lookback_periods = 100, bar_frequency = "1D"):
    """
        Arguements:
            tickers_to_trade -> List of tickers to find support levels for
            lookback_periods -> Number of bars to use when identifying key support levels
            bar_frequency -> Frequency of candles ('5Min', '1D', etc.. Consult Alpaca API )
    """
    key_levels = []

    for ticker in tickers_to_trade:
        df = get_price_dataframe(ticker, bar_frequency, lookback_periods)
        # Getting messy levels, will filter later
        levels = []
        for i in range(2,len(df)-2):
            if isSupport(df,i):
                levels.append((i, df['low'][i], "support"))
            elif isResistance(df,i):
                levels.append((i ,df['high'][i], "resistance"))

        # Average size of candlestick -> using later to check if already identified key levels that are similar
        s =  np.mean(df['high'] - df['low'])

        # Filtering levels to only unique levels
        levels = []
        for i in range(2,len(df)-2):
            if isSupport(df,i):
                l = df['low'][i]
                # Filters out levels close to already discovered levels
                if isFarFromLevel(l, levels, s):
                    levels.append((i,l,"support"))
            elif isResistance(df,i):
                l = df['high'][i]
                # Filters out levels close to already discovered levels
                if isFarFromLevel(l, levels, s):
                    levels.append((i,l,"resistance"))

        support = [level for index, level, category in levels if category == "support"]
        resistance = [level for index, level, category in levels if category == "resistance"]

        # Adding to master list
        key_levels.append({"ticker": ticker, "support": support,"resistance": resistance})
    return key_levels

def find_stocks_at_key_levels(stocks_to_trade, proximity_threshold_percent = 2):
    """
        Arguments:
            stocks_to_trade -> List of stock elements. Each element is a dictionary with keys ('ticker': str, 'support': list, 'resistance': list)
            proximity_threshold_percent -> percentage difference used to judge if current price is close to support level
        
        Returns:
            stocks_at_support -> List of stock elements whose current price is within proximity_threshold_percent of support level
            stocks_at_resistance -> List of stock elements whose current price is within proximity_threshold_percent of resistance level
    """
    # Find if current price is near any of these resistance levels
    stocks_at_support = []
    stocks_at_resistance = []

    for stock in stocks_to_trade:
        # Get current price
        quote = api.get_last_quote(stock["ticker"])
        current_price = np.mean([quote.bidprice, quote.askprice])

        # Check if stock is near a support level
        for level in stock["support"]:
            difference = ((current_price - level)/level)*100
            # If stock is within 2% of a support level...
            if difference < proximity_threshold_percent and difference > -proximity_threshold_percent:
                near_support = True
            else:
                near_support = False

        # Check if stock is near a resistance level
        for level in stock["resistance"]:
            difference = ((current_price - level)/level)*100
            # If stock is within 2% of a support level...
            if difference < proximity_threshold_percent and difference > -proximity_threshold_percent:
                near_resistance = True
            else:
                near_resistance = False

        # Adding stocks near a key level to watch list
        if near_support:
            stocks_at_support.append(stock)
        elif near_resistance:
            stocks_at_resistance.append(stock)
            
    return stocks_at_support, stocks_at_resistance

def append_trade_indicators(key_levels_list):
    """
        Arguements:
            key_levels_list => LIST of SUPPORT or RESISTANCE LEVELS of which to compare the current market price
                            => Only the last key level will be compared
        Returns:
            Pandas DataFrame with added trading indicators
    """
    dataframe = get_price_dataframe(ticker)
#     data = dataframe.iloc[-lookback_periods:]
    dataframe.reset_index(inplace=True, drop=True)
    
    for i in range(len(dataframe)-1):
    # Adding change indicators
        # PRICE CHANGE
        if dataframe["close"][i] < dataframe["close"][i+1]:
            dataframe.at[i+1, "price_change"] = "increasing"
        else:
            dataframe.at[i+1, "price_change"] = "decreasing"
        # CANDLE SIZE
        dataframe.at[i+1, "candle_size"] = dataframe["high"][i+1] - dataframe["low"][i+1]
    # AVERAGE VOLUME
    avg = []
    for index, row in dataframe.iterrows():
        if len(avg) == 5:
            avg.pop(0)
        avg.append(row["volume"])
        dataframe.at[index,"avg_5_volumes"] = np.mean(avg)
    return dataframe

# Evaluating trade at resistance
def evaluate_trade_at_resistance(dataframe, resistance_list):
    """
        Arguements:
            dataframe -> Pandas dataframe with trade indicators added (use append_trade_indicators() function)
            resistance_list -> List of key resistance levels to use in calculation
        
        Returns:
            Buy Signal -> Returns 'buy' if you should buy now, returns 'hold' if you should not buy now
                       -> Will not tell you to sell or short the security         
        Evaluates:
            1) If current price above resistance
            2) If current volume is greater than average volume
            3) If 25 min avgerage volume is increasing over the last 15 minutes 
            4) If most recent candle bar size is increasing
            4) If last closing price is increasing 
    """
    ##################### BUY ON BREAKOUT TO THE UPSIDE #####################
    # Checking if current ask price is above resistance
    if api.get_last_quote(ticker).askprice < resistance_list[-1]:
        return "hold"
    # Checking if the current volume is greater than average volume
    avg_volume = np.mean(dataframe["volume"])
    if dataframe.iloc[-1]["volume"] < avg_volume:
        return "hold"
    # Check if increasing avg volume over last 3 bars
    if dataframe.iloc[-1]["avg_5_volumes"] > dataframe.iloc[-2]["avg_5_volumes"] and dataframe.iloc[-2]["avg_5_volumes"] > dataframe.iloc[-3]["avg_5_volumes"] and dataframe.iloc[-2]["avg_5_volumes"] > dataframe.iloc[-3]["avg_5_volumes"]:
        # Check if candle size and price are increasing on last bar
        if dataframe.iloc[-1]["candle_size"] > dataframe.iloc[-2]["candle_size"] and dataframe.iloc[-1]["price_change"] == "increasing":   
            return "buy"
        else:
            return "hold"
    else:
        return "hold"

# Evaluating trade at support
def evaluate_trade_at_support(dataframe, support_list):
    """
        Arguements:
            dataframe -> Pandas dataframe with trade indicators added (use append_trade_indicators() function)
            support_list -> List of key support levels to use in calculation
        
        Returns:
            Sell Signal -> Returns 'sell' if you should short the security now, returns 'hold' if you should not short the security now
                       -> Will not tell you to buy the security   
        Evaluates:
            1) If current price below support
            2) If current volume is greater than average volume
            3) If 25 min avgerage volume is increasing over the last 15 minutes 
            4) If most recent candle bar size is increasing
            4) If last closing price is decreasing 
    """
    ##################### SELL ON BREAKOUT TO THE DOWNSIDE #####################
    # Checking if current ask price is below support
    if api.get_last_quote(ticker).askprice > support_list[-1]:
        return "hold"
    # Checking if the current volume is greater than average volume
    avg_volume = np.mean(dataframe["volume"])
    if dataframe.iloc[-1]["volume"] < avg_volume:
        return "hold"
    # Check if increasing avg volume over last 3 bars
    if dataframe.iloc[-1]["avg_5_volumes"] > dataframe.iloc[-2]["avg_5_volumes"] and dataframe.iloc[-2]["avg_5_volumes"] > dataframe.iloc[-3]["avg_5_volumes"] and dataframe.iloc[-2]["avg_5_volumes"] > dataframe.iloc[-3]["avg_5_volumes"]:
        # Check if candle size and price are increasing on last bar
        if dataframe.iloc[-1]["candle_size"] > dataframe.iloc[-2]["candle_size"] and dataframe.iloc[-1]["price_change"] == "decreasing":   
            return "sell"
        else:
            return "hold"
    else:
        return "hold"

### Portfolio Object

In [4]:
def is_market_open():
    clock = api.get_clock()
    if clock.is_open:
        return True
    else:
        return False

def is_account_blocked():
    account = api.get_account()
    if account.trading_blocked:
        return True
    else:
        return False
    
def is_ticker_owned_already(ticker):
    positions = api.list_positions()
    for position in positions:
        if position.symbol == ticker:
            qty = int(position.qty)
            if qty > 0:
                return True, qty
            elif qty < 0:
                return True, abs(qty)
    return False, 0

def get_buying_power():
    account = api.get_account()
    buying_power = account.buying_power
    return buying_power

def any_open_positions():
    positions = api.list_positions()
    if positions:
        return True     
    else:
        return False

### Transaction Cost Object

### Risk Object

### Execution Object

In [5]:
import random

def buy_stock(symbol='SPY',qty=1,type='market',time_in_force='gtc'):
    order_number = str(random.randint(100000000000,999999999999))
    api.submit_order(
        symbol=symbol,
        qty=qty,
        side='buy',
        type=type,
        time_in_force=time_in_force,
        client_order_id=order_number)
    return order_number

def sell_stock(symbol='SPY',qty=1,type='market',time_in_force='gtc'):
    order_number = str(random.randint(100000000000,999999999999))
    api.submit_order(
        symbol=symbol,
        qty=qty,
        side='sell',
        type=type,
        time_in_force=time_in_force,
        client_order_id=order_number)
    return order_number

## Current Strategy: Momentum
#### Breakout
    * Look at past highs and generate buy signals as soon as price manages to break out from its previous high
    * Use dynamic lookback ranges based on volatility
        * When volatility is high, look further back
        * When low, look less
    * Open short-term long position and use stop loss
#### Gap and Go
    * Looks for overnight gappers and bets on continuation of that move
    * Open Short-term long trade and use stop loss
#### Earnings/News
    * Look at the most well known/liquid stocks that had earnings in the past few days
    * Focus on stocks that have an especially negative reaction from the market (-2%)
    * Open long position and use stop loss

## Algorithm Structure
#### Universe Selection
    * Which asset class? (US Stocks)
    * Filters
        * Liquidity (Top 10 by dollar Volume)
        * Price (Too high or too low)
        * Industry (Tech Stocks)
#### Indicators
    * Calculate trade signal
    * Assign alpha scores to the securities that were passed from universe selection model
    * Example: Directional assumption of security and confidence in prediction
#### Portfolio 
    * Where trade signals from indicators are turned into a potential portfolio
    * Decides how much capital to allocate to each position
        * Same amount capital allocated to each trade signal
        * Can allocate more to a specific trade based upon higher levels of confidence if known
        * Doesnt allocate more than 75% of available funds
#### Transaction Cost
    * Account for transaction costs
    * Can be included in execution or portfolio model
    * Calculates
        * Commissions
        * Slippage
        * Market Impact
#### Risk
    * Monitors portfolios risk, manages positions when necessary
    * Example: Closes a position if down a certain percentage
    * If the position P/L > 5% or P/L < -2% close position
#### Execution
    * Order size, order type, order time, order price, etc.
    * Split large orders up into smaller orders over time?
    * Market, limit or other order type
    * Called every monday @ 10:00am
        * Opens or closes positions based on information it recieves from portfolio and risk model    

### Use This Area To Test Algorithm Improvements

In [6]:
# Finding the most liquid stocks that fit our criteria
stocks_to_trade_long = find_most_liquid_stocks(periods=10, num_returned=100, bar_frequency="5Min")

# Calculating key levels of stocks to evaluate which stocks to trade
key_levels = get_key_levels(stocks_to_trade_long, lookback_periods=21, bar_frequency='1D')

# Filter out all stocks without calculated support or resistance levels
stocks_to_trade = [stock for stock in key_levels if stock["support"] and stock["resistance"]]

# Find stocks currently near a key level
stocks_at_support, stocks_at_resistance = find_stocks_at_key_levels(stocks_to_trade, proximity_threshold_percent=2)

In [16]:
# Evaluate stocks at resistance for buy signal
for stock in stocks_at_resistance:
    ticker = stock["ticker"]
    resistance = stock["resistance"]
    
    dataframe = append_trade_indicators(resistance)
    
    buy_signal = evaluate_trade_at_resistance(dataframe, resistance)
    print(f"{ticker}: {buy_signal}")

SWKS: buy
NATI: hold
JAMF: hold
VMW: buy
PTC: hold
INTU: hold
AL: hold
ADP: hold
IBM: buy
MA: hold
IPG: buy
TEL: hold
QCOM: hold
ATVI: hold
VNT: hold


In [18]:
# Evaluate stocks at support for sell signal
for stock in stocks_at_support:
    ticker = stock["ticker"]
    support = stock["support"]
    
    dataframe = append_trade_indicators(support)
    
    buy_signal = evaluate_trade_at_support(dataframe, support)
    print(f"{ticker}: {buy_signal}")

CLGX: hold
XRX: hold
TDC: hold
MDLA: hold
AMAT: hold
NVDA: hold
XLNX: hold
V: hold
SNAP: hold


In [19]:
ticker = "V"
append_trade_indicators(stocks_at_support[-2]["support"])

,date,open,high,low,close,volume,price_change,candle_size,avg_5_volumes
0,2020-12-23 11:10:00-05:00,206.260,206.305,206.140,206.305,3231,NaN,NaN,3231.000000
1,2020-12-23 11:15:00-05:00,206.310,206.390,206.220,206.220,4151,decreasing,0.170,3691.000000
2,2020-12-23 11:20:00-05:00,206.330,206.380,206.285,206.310,4739,increasing,0.095,4040.333333
3,2020-12-23 11:25:00-05:00,206.350,206.410,206.240,206.240,5269,decreasing,0.170,4347.500000
4,2020-12-23 11:30:00-05:00,206.220,206.460,206.180,206.345,3860,increasing,0.280,4250.000000
...,...,...,...,...,...,...,...,...,...
95,2020-12-24 12:35:00-05:00,208.660,208.660,208.460,208.495,2567,decreasing,0.200,2578.800000
96,2020-12-24 12:40:00-05:00,208.520,208.550,208.360,208.550,3099,increasing,0.190,2788.000000
97,2020-12-24 12:45:00-05:00,208.550,208.580,208.460,208.570,3499,increasing,0.120,3192.200000
98,2020-12-24 12:50:00-05:00,208.600,208.635,208.530,208.580,5341,increasing,0.105,3510.600000


In [20]:
stocks_at_support[-2]

{'ticker': 'V', 'support': [207.61], 'resistance': [211.98]}

## TO DO
- Add filter if number of buy recommendations is over a certain number?
- Find 'best buy' out of buy recommendations?
- Look for sideways charts to eliminate flat trading stocks? (use V method)?
- Add functionality to evaluate stock at support for buy signal (more strict, dont want a lot of buy signals)
- Add functionality to evaluate stock at resistance for sell signal (more strict, dont want a lot of buy signals)

### Next Milestone in Project

1) Identify the Candidate: Find stocks that have built strong support or resistance levels and watch them. Remember, the stronger the support or resistance, the better the outcome. Make sure you understand this when you shop for stocks.

2) Wait for the Breakout: Finding a good candidate does not mean a trade should be taken prematurely. Wait patiently for the stock price to make its move. To be sure the breakout will hold, on the day the stock price trades outside its support or resistance level, wait until near the end of the trading day to make your move.

3) Set a Reasonable Objective: If you are going to take a trade, set an expectation of where it is going. If you don't, you won't know where to exit the trade. This can be done by calculating an average move the stock makes or measuring the distance between support and resistance (especially when trading price patterns).

4) Allow the Stock to Retest: This is the most critical step. When a stock price breaks a resistance level, old resistance becomes new support. When a stock breaks a support level, old support becomes new resistance. In the majority of your trades, the stock will test the level it has broken after the first couple of days. Prepare for it.

5) Know When Your Trade/Pattern Has Failed: When the stock attempts to retest a prior support or resistance level and it breaks back through it, this is where a pattern or breakout has failed. It is imperative you take the loss at this point. Don't gamble with your losses.

6) Exit Trades Toward the Market Close: You can't discern at the open whether prices will hold at a particular level. This is why you might consider waiting until near the market close to exit a losing trade. If a stock has remained outside a predetermined support or resistance level toward the market close, it is time to close the position and move on to the next.

7) Be Patient: This strategy requires plenty of patience. By following these steps, you will reduce emotion and be more objective about a trade.

8) Exit at Your Target: If you are not exiting the trade with a loss, then you are in the trade. You should remain in the trade until the stock price reaches its objective or you reach your time target without hitting your target price.

In [ ]:
# while market_open():
    # Check to see if stock is breaking out
        # Breakout if asset's price moves above a resistance level or moves below a support level on increasing volume.
        # Confirmation: Many investors look for above-average volume as confirmation
            # OR wait toward the close of a trading period to determine whether prices will sustain the levels they've broken out of.
        
    # If breaking out, send trade signal
        # Include exit objective
        # Old resistance becomes new support, old support becomes new resistance
        # Allow stock to retest new support
            # If breaks through prior support or resistance, exit position
            # Only exit losing trades at end of market
        # Remain in the trade until stock price reaches its objective or you reach your time target without hitting your target price
        
     # Save exit objective and other criticle information to DB for all positions holding overnight in portfolio
     